# Consolidando o resultado do modelo com variáveis importantes para negócio em um Produto de Dados Analítico

### Importando as bibliotecas & Carregando os dados
---

In [0]:
from pyspark.sql.functions import date_format, when, col, hour, concat_ws, month, dayofmonth, count, avg, row_number, udf, concat, sum, max, round, dayofweek, year, lit, cast, current_date
from pyspark.sql.types import StringType, TimestampType
from pyspark.sql.window import Window

In [0]:
tabela_bronze = "estudo.default.tvendas_bronze"
tabela_cluster_silver = "estudo.default.tcluster_silver"
tabela_final = "estudo.default.tcluster_cli_gold"

In [0]:
df_origem = spark.table(tabela_bronze)
df_cluster = spark.table(tabela_cluster_silver)

### Transformando e Agregando os dados
---

In [0]:
df_enriquecido = df_origem.withColumn("datetime_purchase",
                 concat_ws(" ", col("date_purchase"), col("time_purchase")).cast("timestamp")) \
    .withColumn("date_purchase", col("date_purchase").cast("date")) \
    .withColumn("ano_mes_compra", date_format(col("date_purchase"), "yyyy-MM")) \
    .withColumn("hora_do_dia_compra", hour(col("time_purchase"))) \
    .withColumn("dia_semana_compra", dayofweek(col("date_purchase"))) \
    .withColumn("rota_ida", concat_ws(" -> ", col("place_origin_departure"), col("place_destination_departure"))) \
    .withColumn("rota_volta",
                   when(col("place_origin_return").isNotNull(),
                        concat_ws(" -> ", col("place_origin_return"), col("place_destination_return")))
                   .otherwise(None)) \
    .withColumn("mes_compra", month(col("date_purchase"))) \
    .withColumn("dia_do_mes_compra", dayofmonth(col("date_purchase"))) \
    .withColumn("estacao_do_ano",
        when((month(col("date_purchase")) == 12) & (dayofmonth(col("date_purchase")) >= 21) | \
            (month(col("date_purchase")) == 1) | \
            (month(col("date_purchase")) == 2) | \
            (month(col("date_purchase")) == 3) & (dayofmonth(col("date_purchase")) < 20), "Verão")
        .when((month(col("date_purchase")) == 3) & (dayofmonth(col("date_purchase")) >= 20) | \
            (month(col("date_purchase")) == 4) | \
            (month(col("date_purchase")) == 5) | \
            (month(col("date_purchase")) == 6) & (dayofmonth(col("date_purchase")) < 20), "Outono")
        .when((month(col("date_purchase")) == 6) & (dayofmonth(col("date_purchase")) >= 20) | \
            (month(col("date_purchase")) == 7) | \
            (month(col("date_purchase")) == 8) | \
            (month(col("date_purchase")) == 9) & (dayofmonth(col("date_purchase")) < 22), "Inverno")
        .when((month(col("date_purchase")) == 9) & (dayofmonth(col("date_purchase")) >= 22) | \
            (month(col("date_purchase")) == 10) | \
            (month(col("date_purchase")) == 11) | \
            (month(col("date_purchase")) == 12) & (dayofmonth(col("date_purchase")) < 21), "Primavera")
        .otherwise("Não Definido")
    )

In [0]:
df_metricas_agrupadas = df_enriquecido.groupBy("fk_contact").agg(
    count(col("nk_ota_localizer_id")).alias("total_compras"),
    round(sum(col("gmv_success")), 2).alias("valor_total_compra"),
    round(avg(col("gmv_success")), 2).alias("valor_medio_compra"),
    sum(col("total_tickets_quantity_success")).alias("total_tickets_compras"),
    sum(when(col("rota_volta").isNotNull(), 1).otherwise(0)).alias("total_compras_com_volta"),
    max(col("datetime_purchase")).alias("data_ultima_compra")
)

In [0]:
def calculando_info_preferida(dataframe, group_cols, target_col, output_col_name):
    window_spec_count = Window.partitionBy(*group_cols, target_col)

    df_counted = dataframe.withColumn(
        "count_target",
        count(target_col).over(window_spec_count)
    )

    window_spec_rank = Window.partitionBy(*group_cols).orderBy(col("count_target").desc(), col(target_col))

    df_mode = df_counted.withColumn(
        "rank_mode",
        row_number().over(window_spec_rank)
    ).filter(col("rank_mode") == 1) \
     .select(*group_cols, col(target_col).alias(output_col_name))

    return df_mode

In [0]:
rota_ida_frequente = calculando_info_preferida(
    df_enriquecido,
    ["fk_contact"],
    "rota_ida",
    "local_partida_ida_local_destino_ida_frequente"
)

rota_volta_frequente = calculando_info_preferida(
    df_enriquecido.filter(col("rota_volta").isNotNull()),
    ["fk_contact"],
    "rota_volta",
    "local_partida_volta_local_destino_volta_frequente"
)

dia_preferido = calculando_info_preferida(
    df_enriquecido,
    ["fk_contact"],
    "dia_semana_compra",
    "dia_da_semana_preferido_de_compra"
)

periodo_dia_preferido = calculando_info_preferida(
    df_enriquecido,
    ["fk_contact"],
    "hora_do_dia_compra",
    "periodo_do_dia_preferido_para_compra"
)

epoca_ano_preferida = calculando_info_preferida(
    df_enriquecido,
    ["fk_contact"],
    "mes_compra",
    "epoca_do_ano_preferido_da_compra"
)

epoca_mes_preferida = calculando_info_preferida(
    df_enriquecido,
    ["fk_contact"],
    "dia_do_mes_compra",
    "epoca_do_mes_preferido_da_compra"
)

estacao_do_ano_preferida = calculando_info_preferida(
    df_enriquecido,
    ["fk_contact"],
    "estacao_do_ano",
    "estacao_do_ano_preferido_da_compra"
)

In [0]:
df_metricas_final = df_metricas_agrupadas \
    .join(rota_ida_frequente, on=["fk_contact"], how="left") \
    .join(rota_volta_frequente, on=["fk_contact"], how="left") \
    .join(dia_preferido, on=["fk_contact"], how="left") \
    .join(periodo_dia_preferido, on=["fk_contact"], how="left") \
    .join(epoca_ano_preferida, on=["fk_contact"], how="left") \
    .join(epoca_mes_preferida, on=["fk_contact"], how="left") \
    .join(estacao_do_ano_preferida, on=["fk_contact"], how="left")

In [0]:
df_calculado = df_metricas_final.withColumn("media_tickets_por_compra", round(col("total_tickets_compras") / col("total_compras"), 2)) \
    .withColumn("porcentagem_viagens_com_volta", round(col("total_compras_com_volta") / col("total_compras") * 100, 2))

In [0]:
df_join = df_calculado \
    .join(df_cluster, on=["fk_contact"], how="left")

In [0]:
df_final = df_join.select(
        "fk_contact",
        col("cluster").alias("cd_cluster"),
        col("segmento_do_cliente").alias("ds_cluster"),
        col("recencia_dias").alias("nu_recencia_dias"),
        round("valor_total_gasto", 2).alias("vl_total"),
        round("valor_medio_compra", 2).alias("vl_medio"),
        col("frequencia_de_compras").alias("qt_compra"),
        col("total_tickets_compras").alias("qt_total_ticket"),
        col("media_tickets_por_compra").alias("qt_media_ticket"),
        col("total_compras_com_volta").alias("qt_total_compra_volta"),
        col("porcentagem_viagens_com_volta").alias("pc_compra_volta"),
        col("local_partida_ida_local_destino_ida_frequente").alias("ds_rota_ida_preferido"),
        col("local_partida_volta_local_destino_volta_frequente").alias("ds_rota_volta_preferido"),
        when(col("dia_da_semana_preferido_de_compra") == 1, 'Domingo')
            .when(col("dia_da_semana_preferido_de_compra") == 2, 'Segunda')
            .when(col("dia_da_semana_preferido_de_compra") == 3, 'Terça')
            .when(col("dia_da_semana_preferido_de_compra") == 4, 'Quarta')
            .when(col("dia_da_semana_preferido_de_compra") == 5, 'Quinta')
            .when(col("dia_da_semana_preferido_de_compra") == 6, 'Sexta')
            .when(col("dia_da_semana_preferido_de_compra") == 7, 'Sábado').alias("ds_dia_semana_preferido"),
        col("epoca_do_mes_preferido_da_compra").alias("ds_dia_mes_preferido"),
        col("periodo_do_dia_preferido_para_compra").alias("nu_hora_preferido"),
        when((col("periodo_do_dia_preferido_para_compra") >= 0) & (col("periodo_do_dia_preferido_para_compra") <= 5), "Madrugada")
                   .when((col("periodo_do_dia_preferido_para_compra") >= 6) & (col("periodo_do_dia_preferido_para_compra") <= 11), "Manhã")
                   .when((col("periodo_do_dia_preferido_para_compra") >= 12) & (col("periodo_do_dia_preferido_para_compra") <= 17), "Tarde")
                   .otherwise("Noite").alias("ds_periodo_dia_preferido"),
        when(col("epoca_do_ano_preferido_da_compra") == 1, 'Janeiro')
            .when(col("epoca_do_ano_preferido_da_compra") == 2, 'Fevereiro')
            .when(col("epoca_do_ano_preferido_da_compra") == 3, 'Março')
            .when(col("epoca_do_ano_preferido_da_compra") == 4, 'Abril')
            .when(col("epoca_do_ano_preferido_da_compra") == 5, 'Maio')
            .when(col("epoca_do_ano_preferido_da_compra") == 6, 'Junho')
            .when(col("epoca_do_ano_preferido_da_compra") == 7, 'Julho')
            .when(col("epoca_do_ano_preferido_da_compra") == 8, 'Agosto')
            .when(col("epoca_do_ano_preferido_da_compra") == 9, 'Setembro')
            .when(col("epoca_do_ano_preferido_da_compra") == 10, 'Outubro')
            .when(col("epoca_do_ano_preferido_da_compra") == 11, 'Novembro')
            .when(col("epoca_do_ano_preferido_da_compra") == 12, 'Dezembro').alias("ds_mes_preferido"),
        col("estacao_do_ano_preferido_da_compra").alias("ds_estacao_preferido"),
        current_date().alias("dt_particao")
    )

### Escrita na tabela relacional
---

In [0]:
df_final.write.mode("overwrite").saveAsTable(tabela_final)